#Dataset Preparation

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

In [15]:
data=pd.read_csv('dataset-big.csv')
# Dropping unneccesary columns
data = data.drop(['filename'],axis=1)
#Encoding the Labels
raga_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(raga_list)
#Scaling the Feature columns
scaler = MinMaxScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))
#Dividing data into training and Testing set
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2)
data.groupby("raga").size()

raga
Purvikalyani         240
abhogi               240
ananda bhairavi      240
atana                200
begada               240
behag                240
bhairavi             240
bilahari             210
darbar               150
hamsadhvani          240
hindolam             250
jaganmohini          150
jaunpuri             140
mayamalava gaula     150
ragamalika           150
shanmukhapriya       160
varali               150
vasanta              150
yadukula kambhoji    150
yamuna kalyani       150
dtype: int64

In [16]:
print(y)

[ 1  1  1 ... 13 13 13]


In [4]:
D_train = xgb.DMatrix(X_train, label=Y_train)
D_test = xgb.DMatrix(X_test, label=Y_test)

In [5]:
param = {
    'eta': 0.04, 
    'max_depth': 3,  
    'objective': 'multi:softprob',  
    'num_class': 20,                  #Change when number of raga changes
    'base_score': 0.5,
    'booster':'gbtree',
    'colsample_bylevel':1,
    'colsample_bynode':1, 
    'colsample_bytree':0.7, 
    'gamma':0.0,
    'learning_rate':0.1, 
    'min_child_weight':1, 
    'n_estimators':100, 
    'n_jobs':1,
} 

steps = 20  # The number of training iterations

In [6]:
model = xgb.train(param, D_train, steps)

/home/sypher/.local/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [14:32:59] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


In [7]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, accuracy_score

preds = model.predict(D_test)
best_preds = np.asarray([np.argmax(line) for line in preds])

print("Precision = {}".format(precision_score(Y_test, best_preds, average='macro')))
print("Recall = {}".format(recall_score(Y_test, best_preds, average='macro')))
print("Accuracy = {}".format(accuracy_score(Y_test, best_preds)))

Precision = 0.9818544138507856
Recall = 0.9814230943160963
Accuracy = 0.9856770833333334


In [8]:
import librosa
songname="carnatic/Kanakadurga Venkatesh at Arkay by Kanakadurga Venkatesh/Lokavana Chatura/Lokavana Chatura.mp3.mp3"
x, sr = librosa.load(songname, mono=True, duration=30)
rmse = librosa.feature.rms(y=x)[0]
chroma_stft = librosa.feature.chroma_stft(y=x, sr=sr)
spec_cent = librosa.feature.spectral_centroid(y=x, sr=sr)
spec_bw = librosa.feature.spectral_bandwidth(y=x, sr=sr)
rolloff = librosa.feature.spectral_rolloff(y=x, sr=sr)
zcr = librosa.feature.zero_crossing_rate(x)
mfcc = librosa.feature.mfcc(y=x, sr=sr)

data=[0,np.mean(rmse),np.mean(chroma_stft),np.mean(spec_cent),np.mean(spec_bw),np.mean(rolloff),np.mean(zcr),np.mean(mfcc[0]),np.mean(mfcc[1]),np.mean(mfcc[2]),np.mean(mfcc[3]),np.mean(mfcc[4]),np.mean(mfcc[5]),np.mean(mfcc[6]),np.mean(mfcc[7]),np.mean(mfcc[8]),np.mean(mfcc[9]),np.mean(mfcc[10]),np.mean(mfcc[11]),np.mean(mfcc[12]),np.mean(mfcc[13]),np.mean(mfcc[14]),np.mean(mfcc[15]),np.mean(mfcc[16]),np.mean(mfcc[17]),np.mean(mfcc[18]),np.mean(mfcc[19])]
columns=['ind','rmse','chroma_stft','spec_cent','spec_bw','rolloff','zcr','mfcc0','mfcc1','mfcc2','mfcc3','mfcc4','mfcc5','mfcc6','mfcc7','mfcc8','mfcc9','mfcc10','mfcc11','mfcc12','mfcc13','mfcc14','mfcc15','mfcc16','mfcc17','mfcc18','mfcc19']
dataset=pd.DataFrame(columns=columns)
dataseries = pd.Series(data, index = dataset.columns)
dataset = dataset._append(dataseries, ignore_index=True)

example = scaler.transform(np.array(dataset.iloc[:, :], dtype = float))
example=xgb.DMatrix(example)
preds = model.predict(example)
best_preds = np.asarray([np.argmax(line) for line in preds])

raganame=set()
for i in range(len(raga_list)):
  raganame.add((raga_list[i],y[i]))
raganame=list(raganame)
raganame.sort()
print(best_preds)
print("Raga for this song is : "+raganame[best_preds[0]][0])

[1]
Raga for this song is : abhogi


/tmp/ipykernel_19496/3154072433.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataset = dataset._append(dataseries, ignore_index=True)


# Logistic Regression + Pred


In [9]:
logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
print('Accuracy of LogisticRegress classifier on training set: {:.3f}'.format(logreg.score(X_train, Y_train)))
print('Accuracy of LogisticRegress classifier on test set: {:.3f}'.format(logreg.score(X_test, Y_test)))

Accuracy of LogisticRegress classifier on training set: 0.665
Accuracy of LogisticRegress classifier on test set: 0.542


In [10]:
import librosa
songname='/content/hindolam.mp3'
x, sr = librosa.load(songname, mono=True, duration=30)
rmse = librosa.feature.rmse(y=x)[0]
chroma_stft = librosa.feature.chroma_stft(y=x, sr=sr)
spec_cent = librosa.feature.spectral_centroid(y=x, sr=sr)
spec_bw = librosa.feature.spectral_bandwidth(y=x, sr=sr)
rolloff = librosa.feature.spectral_rolloff(y=x, sr=sr)
zcr = librosa.feature.zero_crossing_rate(x)
mfcc = librosa.feature.mfcc(y=x, sr=sr)

data=[0,np.mean(rmse),np.mean(chroma_stft),np.mean(spec_cent),np.mean(spec_bw),np.mean(rolloff),np.mean(zcr),np.mean(mfcc[0]),np.mean(mfcc[1]),np.mean(mfcc[2]),np.mean(mfcc[3]),np.mean(mfcc[4]),np.mean(mfcc[5]),np.mean(mfcc[6]),np.mean(mfcc[7]),np.mean(mfcc[8]),np.mean(mfcc[9]),np.mean(mfcc[10]),np.mean(mfcc[11]),np.mean(mfcc[12]),np.mean(mfcc[13]),np.mean(mfcc[14]),np.mean(mfcc[15]),np.mean(mfcc[16]),np.mean(mfcc[17]),np.mean(mfcc[18]),np.mean(mfcc[19])]
columns=['ind','rmse','chroma_stft','spec_cent','spec_bw','rolloff','zcr','mfcc0','mfcc1','mfcc2','mfcc3','mfcc4','mfcc5','mfcc6','mfcc7','mfcc8','mfcc9','mfcc10','mfcc11','mfcc12','mfcc13','mfcc14','mfcc15','mfcc16','mfcc17','mfcc18','mfcc19']
dataset=pd.DataFrame(columns=columns)
dataseries = pd.Series(data, index = dataset.columns)
dataset = dataset.append(dataseries, ignore_index=True)

example = scaler.transform(np.array(dataset.iloc[:, :], dtype = float))
preds = logreg.predict(example)
best_preds = np.asarray([np.argmax(line) for line in preds])

raganame=set()
for i in range(len(raga_list)):
  raganame.add((raga_list[i],y[i]))
raganame=list(raganame)
raganame.sort()
print(preds)
print("Raga for this song is : "+raganame[preds[0]][0])

/tmp/ipykernel_19496/3165848628.py:3: UserWarning: PySoundFile failed. Trying audioread instead.
  x, sr = librosa.load(songname, mono=True, duration=30)
/home/sypher/.local/lib/python3.10/site-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


FileNotFoundError: [Errno 2] No such file or directory: '/content/hindolam.mp3'

# Naive Bayes + Pred


In [4]:
from sklearn.preprocessing import MinMaxScaler

y = encoder.fit_transform(raga_list)
#Scaling the Feature columns
scaler = MinMaxScaler()
X = scaler.fit_transform(np.array(data.iloc[:,:-1], dtype = float))
#Dividing data into training and Testing set
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2)
modelNB = MultinomialNB()   
modelNB.fit(X_train, Y_train)
print('Accuracy of NaiveBayes classifier on training set: {:.2f}'.format(modelNB.score(X_train, Y_train)))
print('Accuracy of NaiveBayes classifier on test set: {:.2f}'.format(modelNB.score(X_test, Y_test)))

ValueError: could not convert string to float: 'abhogi'

In [ ]:
import librosa
songname='carnatic/Kanakadurga Venkatesh at Arkay by Kanakadurga Venkatesh/Lokavana Chatura/Lokavana Chatura.mp3.mp3'
x, sr = librosa.load(songname, mono=True, duration=30)
rmse = librosa.feature.rmse(y=x)[0]
chroma_stft = librosa.feature.chroma_stft(y=x, sr=sr)
spec_cent = librosa.feature.spectral_centroid(y=x, sr=sr)
spec_bw = librosa.feature.spectral_bandwidth(y=x, sr=sr)
rolloff = librosa.feature.spectral_rolloff(y=x, sr=sr)
zcr = librosa.feature.zero_crossing_rate(x)
mfcc = librosa.feature.mfcc(y=x, sr=sr)

data=[0,np.mean(rmse),np.mean(chroma_stft),np.mean(spec_cent),np.mean(spec_bw),np.mean(rolloff),np.mean(zcr),np.mean(mfcc[0]),np.mean(mfcc[1]),np.mean(mfcc[2]),np.mean(mfcc[3]),np.mean(mfcc[4]),np.mean(mfcc[5]),np.mean(mfcc[6]),np.mean(mfcc[7]),np.mean(mfcc[8]),np.mean(mfcc[9]),np.mean(mfcc[10]),np.mean(mfcc[11]),np.mean(mfcc[12]),np.mean(mfcc[13]),np.mean(mfcc[14]),np.mean(mfcc[15]),np.mean(mfcc[16]),np.mean(mfcc[17]),np.mean(mfcc[18]),np.mean(mfcc[19])]
columns=['ind','rmse','chroma_stft','spec_cent','spec_bw','rolloff','zcr','mfcc0','mfcc1','mfcc2','mfcc3','mfcc4','mfcc5','mfcc6','mfcc7','mfcc8','mfcc9','mfcc10','mfcc11','mfcc12','mfcc13','mfcc14','mfcc15','mfcc16','mfcc17','mfcc18','mfcc19']
dataset=pd.DataFrame(columns=columns)
dataseries = pd.Series(data, index = dataset.columns)
dataset = dataset.append(dataseries, ignore_index=True)

example = scaler.transform(np.array(dataset.iloc[:, :], dtype = float))
preds = modelNB.predict(example)
#best_preds = np.asarray([np.argmax(line) for line in preds])

raganame=set()
for i in range(len(raga_list)):
  raganame.add((raga_list[i],y[i]))
raganame=list(raganame)
raganame.sort()
print(preds)
print("Raga for this song is : "+raganame[preds[0]][0])

[1]
Raga for this song is : abhogi


In [20]:
print(raganame)

[('Purvikalyani', 0), ('abhogi', 1), ('ananda bhairavi', 2), ('atana', 3), ('begada', 4), ('behag', 5), ('bhairavi', 6), ('bilahari', 7), ('darbar', 8), ('hamsadhvani', 9), ('hindolam', 10), ('jaganmohini', 11), ('jaunpuri', 12), ('mayamalava gaula', 13), ('ragamalika', 14), ('shanmukhapriya', 15), ('varali', 16), ('vasanta', 17), ('yadukula kambhoji', 18), ('yamuna kalyani', 19)]


# Random Forest + Preds

In [17]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
rf.fit(X_train, Y_train);

In [18]:
print('Accuracy of RandomForest classifier on training set: {:.2f}'.format(rf.score(X_train, Y_train)))
print('Accuracy of RandomForest classifier on training set: {:.2f}'.format(rf.score(X_test, Y_test)))


Accuracy of RandomForest classifier on training set: 1.00
Accuracy of RandomForest classifier on training set: 1.00


In [19]:
songname='carnatic/Kanakadurga Venkatesh at Arkay by Kanakadurga Venkatesh/Lokavana Chatura/Lokavana Chatura.mp3.mp3'
x, sr = librosa.load(songname, mono=True, duration=30)
rmse = librosa.feature.rms(y=x)[0]
chroma_stft = librosa.feature.chroma_stft(y=x, sr=sr)
spec_cent = librosa.feature.spectral_centroid(y=x, sr=sr)
spec_bw = librosa.feature.spectral_bandwidth(y=x, sr=sr)
rolloff = librosa.feature.spectral_rolloff(y=x, sr=sr)
zcr = librosa.feature.zero_crossing_rate(x)
mfcc = librosa.feature.mfcc(y=x, sr=sr)

data=[0,np.mean(rmse),np.mean(chroma_stft),np.mean(spec_cent),np.mean(spec_bw),np.mean(rolloff),np.mean(zcr),np.mean(mfcc[0]),np.mean(mfcc[1]),np.mean(mfcc[2]),np.mean(mfcc[3]),np.mean(mfcc[4]),np.mean(mfcc[5]),np.mean(mfcc[6]),np.mean(mfcc[7]),np.mean(mfcc[8]),np.mean(mfcc[9]),np.mean(mfcc[10]),np.mean(mfcc[11]),np.mean(mfcc[12]),np.mean(mfcc[13]),np.mean(mfcc[14]),np.mean(mfcc[15]),np.mean(mfcc[16]),np.mean(mfcc[17]),np.mean(mfcc[18]),np.mean(mfcc[19])]
columns=['ind','rmse','chroma_stft','spec_cent','spec_bw','rolloff','zcr','mfcc0','mfcc1','mfcc2','mfcc3','mfcc4','mfcc5','mfcc6','mfcc7','mfcc8','mfcc9','mfcc10','mfcc11','mfcc12','mfcc13','mfcc14','mfcc15','mfcc16','mfcc17','mfcc18','mfcc19']
dataset=pd.DataFrame(columns=columns)
dataseries = pd.Series(data, index = dataset.columns)
dataset = dataset._append(dataseries, ignore_index=True)

example = scaler.transform(np.array(dataset.iloc[:, :], dtype = float))
preds = rf.predict(example)
best_preds = np.asarray([np.argmax(line) for line in preds])

raganame=set()
for i in range(len(raga_list)):
  raganame.add((raga_list[i],y[i]))
raganame=list(raganame)
raganame.sort()
print(best_preds)
print(preds)
print("Raga for this song is : "+raganame[best_preds[0]][0])

[0]
[1.]
Raga for this song is : Purvikalyani


/tmp/ipykernel_19809/621777137.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataset = dataset._append(dataseries, ignore_index=True)


#Dataset Preparation

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

In [ ]:
data=pd.read_csv('dataset-nosource.csv')
# Dropping unneccesary columns
data = data.drop(['filename'],axis=1)
#Encoding the Labels
raga_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(raga_list)
#Scaling the Feature columns
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))
#Dividing data into training and Testing set
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
print(y)

[0 0 0 ... 5 5 5]


#XGBoost + Pred

In [ ]:
!pip install xgboost

In [ ]:
D_train = xgb.DMatrix(X_train, label=Y_train)
D_test = xgb.DMatrix(X_test, label=Y_test)

In [ ]:
param = {
    'eta': 0.04, 
    'max_depth': 3,  
    'objective': 'multi:softprob',  
    'num_class': 10,                  #Change when number of raga changes
    'base_score': 0.5,
    'booster':'gbtree',
    'colsample_bylevel':1,
    'colsample_bynode':1, 
    'colsample_bytree':0.7, 
    'gamma':0.0,
    'learning_rate':0.1, 
    'min_child_weight':1, 
    'n_estimators':100, 
    'n_jobs':1,
} 

steps = 20  # The number of training iterations

In [ ]:
model = xgb.train(param, D_train, steps)

In [ ]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, accuracy_score

preds = model.predict(D_test)
best_preds = np.asarray([np.argmax(line) for line in preds])

print("Precision = {}".format(precision_score(Y_test, best_preds, average='macro')))
print("Recall = {}".format(recall_score(Y_test, best_preds, average='macro')))
print("Accuracy = {}".format(accuracy_score(Y_test, best_preds)))

Precision = 0.9962962962962962
Recall = 0.9975609756097562
Accuracy = 0.9966666666666667


In [ ]:
import librosa
songname='/content/abogi.mp3'
x, sr = librosa.load(songname, mono=True, duration=30)
rmse = librosa.feature.rmse(y=x)[0]
chroma_stft = librosa.feature.chroma_stft(y=x, sr=sr)
spec_cent = librosa.feature.spectral_centroid(y=x, sr=sr)
spec_bw = librosa.feature.spectral_bandwidth(y=x, sr=sr)
rolloff = librosa.feature.spectral_rolloff(y=x, sr=sr)
zcr = librosa.feature.zero_crossing_rate(x)
mfcc = librosa.feature.mfcc(y=x, sr=sr)

data=[0,np.mean(rmse),np.mean(chroma_stft),np.mean(spec_cent),np.mean(spec_bw),np.mean(rolloff),np.mean(zcr),np.mean(mfcc[0]),np.mean(mfcc[1]),np.mean(mfcc[2]),np.mean(mfcc[3]),np.mean(mfcc[4]),np.mean(mfcc[5]),np.mean(mfcc[6]),np.mean(mfcc[7]),np.mean(mfcc[8]),np.mean(mfcc[9]),np.mean(mfcc[10]),np.mean(mfcc[11]),np.mean(mfcc[12]),np.mean(mfcc[13]),np.mean(mfcc[14]),np.mean(mfcc[15]),np.mean(mfcc[16]),np.mean(mfcc[17]),np.mean(mfcc[18]),np.mean(mfcc[19])]
columns=['ind','rmse','chroma_stft','spec_cent','spec_bw','rolloff','zcr','mfcc0','mfcc1','mfcc2','mfcc3','mfcc4','mfcc5','mfcc6','mfcc7','mfcc8','mfcc9','mfcc10','mfcc11','mfcc12','mfcc13','mfcc14','mfcc15','mfcc16','mfcc17','mfcc18','mfcc19']
dataset=pd.DataFrame(columns=columns)
dataseries = pd.Series(data, index = dataset.columns)
dataset = dataset.append(dataseries, ignore_index=True)

example = scaler.transform(np.array(dataset.iloc[:, :], dtype = float))
example=xgb.DMatrix(example)
preds = model.predict(example)
best_preds = np.asarray([np.argmax(line) for line in preds])

raganame=set()
for i in range(len(raga_list)):
  raganame.add((raga_list[i],y[i]))
raganame=list(raganame)
raganame.sort()
print(best_preds)
print("Raga for this song is : "+raganame[best_preds[0]][0])

[1]
Raga for this song is : abhogi


#Logistic Regression + Pred


In [ ]:
logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
print('Accuracy of LogisticRegress classifier on training set: {:.3f}'.format(logreg.score(X_train, Y_train)))
print('Accuracy of LogisticRegress classifier on test set: {:.3f}'.format(logreg.score(X_test, Y_test)))

Accuracy of LogisticRegress classifier on training set: 0.837
Accuracy of LogisticRegress classifier on test set: 0.790


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
import librosa
songname='/content/hindolam.mp3'
x, sr = librosa.load(songname, mono=True, duration=30)
rmse = librosa.feature.rmse(y=x)[0]
chroma_stft = librosa.feature.chroma_stft(y=x, sr=sr)
spec_cent = librosa.feature.spectral_centroid(y=x, sr=sr)
spec_bw = librosa.feature.spectral_bandwidth(y=x, sr=sr)
rolloff = librosa.feature.spectral_rolloff(y=x, sr=sr)
zcr = librosa.feature.zero_crossing_rate(x)
mfcc = librosa.feature.mfcc(y=x, sr=sr)

data=[0,np.mean(rmse),np.mean(chroma_stft),np.mean(spec_cent),np.mean(spec_bw),np.mean(rolloff),np.mean(zcr),np.mean(mfcc[0]),np.mean(mfcc[1]),np.mean(mfcc[2]),np.mean(mfcc[3]),np.mean(mfcc[4]),np.mean(mfcc[5]),np.mean(mfcc[6]),np.mean(mfcc[7]),np.mean(mfcc[8]),np.mean(mfcc[9]),np.mean(mfcc[10]),np.mean(mfcc[11]),np.mean(mfcc[12]),np.mean(mfcc[13]),np.mean(mfcc[14]),np.mean(mfcc[15]),np.mean(mfcc[16]),np.mean(mfcc[17]),np.mean(mfcc[18]),np.mean(mfcc[19])]
columns=['ind','rmse','chroma_stft','spec_cent','spec_bw','rolloff','zcr','mfcc0','mfcc1','mfcc2','mfcc3','mfcc4','mfcc5','mfcc6','mfcc7','mfcc8','mfcc9','mfcc10','mfcc11','mfcc12','mfcc13','mfcc14','mfcc15','mfcc16','mfcc17','mfcc18','mfcc19']
dataset=pd.DataFrame(columns=columns)
dataseries = pd.Series(data, index = dataset.columns)
dataset = dataset.append(dataseries, ignore_index=True)

example = scaler.transform(np.array(dataset.iloc[:, :], dtype = float))
preds = logreg.predict(example)
best_preds = np.asarray([np.argmax(line) for line in preds])

raganame=set()
for i in range(len(raga_list)):
  raganame.add((raga_list[i],y[i]))
raganame=list(raganame)
raganame.sort()
print(preds)
print("Raga for this song is : "+raganame[preds[0]][0])

[1]
Raga for this song is : abhogi


#Naive Bayes + Pred


In [11]:
y = encoder.fit_transform(raga_list)
#Scaling the Feature columns
scaler = MinMaxScaler()
X = scaler.fit_transform(np.array(raga_list, dtype = float))
#Dividing data into training and Testing set
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2)
modelNB = MultinomialNB()   
modelNB.fit(X_train, Y_train)
print('Accuracy of NaiveBayes classifier on training set: {:.2f}'.format(modelNB.score(X_train, Y_train)))
print('Accuracy of NaiveBayes classifier on test set: {:.2f}'.format(modelNB.score(X_test, Y_test)))

ValueError: could not convert string to float: 'abhogi'

In [ ]:
import librosa
songname='/content/abogi.mp3'
x, sr = librosa.load(songname, mono=True, duration=30)
rmse = librosa.feature.rmse(y=x)[0]
chroma_stft = librosa.feature.chroma_stft(y=x, sr=sr)
spec_cent = librosa.feature.spectral_centroid(y=x, sr=sr)
spec_bw = librosa.feature.spectral_bandwidth(y=x, sr=sr)
rolloff = librosa.feature.spectral_rolloff(y=x, sr=sr)
zcr = librosa.feature.zero_crossing_rate(x)
mfcc = librosa.feature.mfcc(y=x, sr=sr)

data=[0,np.mean(rmse),np.mean(chroma_stft),np.mean(spec_cent),np.mean(spec_bw),np.mean(rolloff),np.mean(zcr),np.mean(mfcc[0]),np.mean(mfcc[1]),np.mean(mfcc[2]),np.mean(mfcc[3]),np.mean(mfcc[4]),np.mean(mfcc[5]),np.mean(mfcc[6]),np.mean(mfcc[7]),np.mean(mfcc[8]),np.mean(mfcc[9]),np.mean(mfcc[10]),np.mean(mfcc[11]),np.mean(mfcc[12]),np.mean(mfcc[13]),np.mean(mfcc[14]),np.mean(mfcc[15]),np.mean(mfcc[16]),np.mean(mfcc[17]),np.mean(mfcc[18]),np.mean(mfcc[19])]
columns=['ind','rmse','chroma_stft','spec_cent','spec_bw','rolloff','zcr','mfcc0','mfcc1','mfcc2','mfcc3','mfcc4','mfcc5','mfcc6','mfcc7','mfcc8','mfcc9','mfcc10','mfcc11','mfcc12','mfcc13','mfcc14','mfcc15','mfcc16','mfcc17','mfcc18','mfcc19']
dataset=pd.DataFrame(columns=columns)
dataseries = pd.Series(data, index = dataset.columns)
dataset = dataset.append(dataseries, ignore_index=True)

example = scaler.transform(np.array(dataset.iloc[:, :], dtype = float))
preds = modelNB.predict(example)
#best_preds = np.asarray([np.argmax(line) for line in preds])

raganame=set()
for i in range(len(raga_list)):
  raganame.add((raga_list[i],y[i]))
raganame=list(raganame)
raganame.sort()
print(preds)
print("Raga for this song is : "+raganame[preds[0]][0])

[1]
Raga for this song is : abhogi


#Random Forest + Preds

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
rf.fit(X_train, Y_train);

In [ ]:
print('Accuracy of RandomForest classifier on training set: {:.2f}'.format(rf.score(X_train, Y_train)))
print('Accuracy of RandomForest classifier on training set: {:.2f}'.format(rf.score(X_test, Y_test)))


Accuracy of RandomForest classifier on training set: 1.00
Accuracy of RandomForest classifier on training set: 1.00


In [ ]:
import librosa
songname='/content/hindolam.mp3'
x, sr = librosa.load(songname, mono=True, duration=30)
rmse = librosa.feature.rmse(y=x)[0]
chroma_stft = librosa.feature.chroma_stft(y=x, sr=sr)
spec_cent = librosa.feature.spectral_centroid(y=x, sr=sr)
spec_bw = librosa.feature.spectral_bandwidth(y=x, sr=sr)
rolloff = librosa.feature.spectral_rolloff(y=x, sr=sr)
zcr = librosa.feature.zero_crossing_rate(x)
mfcc = librosa.feature.mfcc(y=x, sr=sr)

data=[0,np.mean(rmse),np.mean(chroma_stft),np.mean(spec_cent),np.mean(spec_bw),np.mean(rolloff),np.mean(zcr),np.mean(mfcc[0]),np.mean(mfcc[1]),np.mean(mfcc[2]),np.mean(mfcc[3]),np.mean(mfcc[4]),np.mean(mfcc[5]),np.mean(mfcc[6]),np.mean(mfcc[7]),np.mean(mfcc[8]),np.mean(mfcc[9]),np.mean(mfcc[10]),np.mean(mfcc[11]),np.mean(mfcc[12]),np.mean(mfcc[13]),np.mean(mfcc[14]),np.mean(mfcc[15]),np.mean(mfcc[16]),np.mean(mfcc[17]),np.mean(mfcc[18]),np.mean(mfcc[19])]
columns=['ind','rmse','chroma_stft','spec_cent','spec_bw','rolloff','zcr','mfcc0','mfcc1','mfcc2','mfcc3','mfcc4','mfcc5','mfcc6','mfcc7','mfcc8','mfcc9','mfcc10','mfcc11','mfcc12','mfcc13','mfcc14','mfcc15','mfcc16','mfcc17','mfcc18','mfcc19']
dataset=pd.DataFrame(columns=columns)
dataseries = pd.Series(data, index = dataset.columns)
dataset = dataset.append(dataseries, ignore_index=True)

example = scaler.transform(np.array(dataset.iloc[:, :], dtype = float))
preds = rf.predict(example)
best_preds = np.asarray([np.argmax(line) for line in preds])

raganame=set()
for i in range(len(raga_list)):
  raganame.add((raga_list[i],y[i]))
raganame=list(raganame)
raganame.sort()
print(best_preds)
print(preds)
print("Raga for this song is : "+raganame[best_preds[0]][0])

FileNotFoundError: ignored

# 20 Ragas

#Dataset Preparation

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

In [ ]:
data=pd.read_csv('dataset-big.csv')
# Dropping unneccesary columns
data = data.drop(['filename'],axis=1)
#Encoding the Labels
raga_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(raga_list)
#Scaling the Feature columns
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))
#Dividing data into training and Testing set
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
print(y)

[ 1  1  1 ... 13 13 13]


#XGBoost + Pred

In [ ]:
!pip install xgboost

In [ ]:
D_train = xgb.DMatrix(X_train, label=Y_train)
D_test = xgb.DMatrix(X_test, label=Y_test)

In [ ]:
param = {
    'eta': 0.04, 
    'max_depth': 3,  
    'objective': 'multi:softprob',  
    'num_class': 20,                  #Change when number of raga changes
    'base_score': 0.5,
    'booster':'gbtree',
    'colsample_bylevel':1,
    'colsample_bynode':1, 
    'colsample_bytree':0.7, 
    'gamma':0.0,
    'learning_rate':0.1, 
    'min_child_weight':1, 
    'n_estimators':100, 
    'n_jobs':1,
} 

steps = 20  # The number of training iterations

In [ ]:
model = xgb.train(param, D_train, steps)

In [ ]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, accuracy_score

preds = model.predict(D_test)
best_preds = np.asarray([np.argmax(line) for line in preds])

print("Precision = {}".format(precision_score(Y_test, best_preds, average='macro')))
print("Recall = {}".format(recall_score(Y_test, best_preds, average='macro')))
print("Accuracy = {}".format(accuracy_score(Y_test, best_preds)))

Precision = 0.9959444444444443
Recall = 0.9959565412186379
Accuracy = 0.99609375


In [ ]:
import librosa
songname='/content/abogi.mp3'
x, sr = librosa.load(songname, mono=True, duration=30)
rmse = librosa.feature.rmse(y=x)[0]
chroma_stft = librosa.feature.chroma_stft(y=x, sr=sr)
spec_cent = librosa.feature.spectral_centroid(y=x, sr=sr)
spec_bw = librosa.feature.spectral_bandwidth(y=x, sr=sr)
rolloff = librosa.feature.spectral_rolloff(y=x, sr=sr)
zcr = librosa.feature.zero_crossing_rate(x)
mfcc = librosa.feature.mfcc(y=x, sr=sr)

data=[0,np.mean(rmse),np.mean(chroma_stft),np.mean(spec_cent),np.mean(spec_bw),np.mean(rolloff),np.mean(zcr),np.mean(mfcc[0]),np.mean(mfcc[1]),np.mean(mfcc[2]),np.mean(mfcc[3]),np.mean(mfcc[4]),np.mean(mfcc[5]),np.mean(mfcc[6]),np.mean(mfcc[7]),np.mean(mfcc[8]),np.mean(mfcc[9]),np.mean(mfcc[10]),np.mean(mfcc[11]),np.mean(mfcc[12]),np.mean(mfcc[13]),np.mean(mfcc[14]),np.mean(mfcc[15]),np.mean(mfcc[16]),np.mean(mfcc[17]),np.mean(mfcc[18]),np.mean(mfcc[19])]
columns=['ind','rmse','chroma_stft','spec_cent','spec_bw','rolloff','zcr','mfcc0','mfcc1','mfcc2','mfcc3','mfcc4','mfcc5','mfcc6','mfcc7','mfcc8','mfcc9','mfcc10','mfcc11','mfcc12','mfcc13','mfcc14','mfcc15','mfcc16','mfcc17','mfcc18','mfcc19']
dataset=pd.DataFrame(columns=columns)
dataseries = pd.Series(data, index = dataset.columns)
dataset = dataset.append(dataseries, ignore_index=True)

example = scaler.transform(np.array(dataset.iloc[:, :], dtype = float))
example=xgb.DMatrix(example)
preds = model.predict(example)
best_preds = np.asarray([np.argmax(line) for line in preds])

raganame=set()
for i in range(len(raga_list)):
  raganame.add((raga_list[i],y[i]))
raganame=list(raganame)
raganame.sort()
print(best_preds)
print("Raga for this song is : "+raganame[best_preds[0]][0])

[1]
Raga for this song is : abhogi


#Logistic Regression + Pred


In [ ]:
logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
print('Accuracy of LogisticRegress classifier on training set: {:.3f}'.format(logreg.score(X_train, Y_train)))
print('Accuracy of LogisticRegress classifier on test set: {:.3f}'.format(logreg.score(X_test, Y_test)))

Accuracy of LogisticRegress classifier on training set: 0.651
Accuracy of LogisticRegress classifier on test set: 0.590


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
import librosa
songname='/content/hindolam.mp3'
x, sr = librosa.load(songname, mono=True, duration=30)
rmse = librosa.feature.rmse(y=x)[0]
chroma_stft = librosa.feature.chroma_stft(y=x, sr=sr)
spec_cent = librosa.feature.spectral_centroid(y=x, sr=sr)
spec_bw = librosa.feature.spectral_bandwidth(y=x, sr=sr)
rolloff = librosa.feature.spectral_rolloff(y=x, sr=sr)
zcr = librosa.feature.zero_crossing_rate(x)
mfcc = librosa.feature.mfcc(y=x, sr=sr)

data=[0,np.mean(rmse),np.mean(chroma_stft),np.mean(spec_cent),np.mean(spec_bw),np.mean(rolloff),np.mean(zcr),np.mean(mfcc[0]),np.mean(mfcc[1]),np.mean(mfcc[2]),np.mean(mfcc[3]),np.mean(mfcc[4]),np.mean(mfcc[5]),np.mean(mfcc[6]),np.mean(mfcc[7]),np.mean(mfcc[8]),np.mean(mfcc[9]),np.mean(mfcc[10]),np.mean(mfcc[11]),np.mean(mfcc[12]),np.mean(mfcc[13]),np.mean(mfcc[14]),np.mean(mfcc[15]),np.mean(mfcc[16]),np.mean(mfcc[17]),np.mean(mfcc[18]),np.mean(mfcc[19])]
columns=['ind','rmse','chroma_stft','spec_cent','spec_bw','rolloff','zcr','mfcc0','mfcc1','mfcc2','mfcc3','mfcc4','mfcc5','mfcc6','mfcc7','mfcc8','mfcc9','mfcc10','mfcc11','mfcc12','mfcc13','mfcc14','mfcc15','mfcc16','mfcc17','mfcc18','mfcc19']
dataset=pd.DataFrame(columns=columns)
dataseries = pd.Series(data, index = dataset.columns)
dataset = dataset.append(dataseries, ignore_index=True)

example = scaler.transform(np.array(dataset.iloc[:, :], dtype = float))
preds = logreg.predict(example)
best_preds = np.asarray([np.argmax(line) for line in preds])

raganame=set()
for i in range(len(raga_list)):
  raganame.add((raga_list[i],y[i]))
raganame=list(raganame)
raganame.sort()
print(preds)
print("Raga for this song is : "+raganame[preds[0]][0])

[1]
Raga for this song is : abhogi


#Naive Bayes + Pred


In [ ]:
y = encoder.fit_transform(raga_list)
#Scaling the Feature columns
scaler = MinMaxScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))
#Dividing data into training and Testing set
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2)
modelNB = MultinomialNB()   
modelNB.fit(X_train, Y_train)
print('Accuracy of NaiveBayes classifier on training set: {:.2f}'.format(modelNB.score(X_train, Y_train)))
print('Accuracy of NaiveBayes classifier on test set: {:.2f}'.format(modelNB.score(X_test, Y_test)))

Accuracy of NaiveBayes classifier on training set: 0.23
Accuracy of NaiveBayes classifier on test set: 0.22


In [ ]:
import librosa
songname='/content/abogi.mp3'
x, sr = librosa.load(songname, mono=True, duration=30)
rmse = librosa.feature.rmse(y=x)[0]
chroma_stft = librosa.feature.chroma_stft(y=x, sr=sr)
spec_cent = librosa.feature.spectral_centroid(y=x, sr=sr)
spec_bw = librosa.feature.spectral_bandwidth(y=x, sr=sr)
rolloff = librosa.feature.spectral_rolloff(y=x, sr=sr)
zcr = librosa.feature.zero_crossing_rate(x)
mfcc = librosa.feature.mfcc(y=x, sr=sr)

data=[0,np.mean(rmse),np.mean(chroma_stft),np.mean(spec_cent),np.mean(spec_bw),np.mean(rolloff),np.mean(zcr),np.mean(mfcc[0]),np.mean(mfcc[1]),np.mean(mfcc[2]),np.mean(mfcc[3]),np.mean(mfcc[4]),np.mean(mfcc[5]),np.mean(mfcc[6]),np.mean(mfcc[7]),np.mean(mfcc[8]),np.mean(mfcc[9]),np.mean(mfcc[10]),np.mean(mfcc[11]),np.mean(mfcc[12]),np.mean(mfcc[13]),np.mean(mfcc[14]),np.mean(mfcc[15]),np.mean(mfcc[16]),np.mean(mfcc[17]),np.mean(mfcc[18]),np.mean(mfcc[19])]
columns=['ind','rmse','chroma_stft','spec_cent','spec_bw','rolloff','zcr','mfcc0','mfcc1','mfcc2','mfcc3','mfcc4','mfcc5','mfcc6','mfcc7','mfcc8','mfcc9','mfcc10','mfcc11','mfcc12','mfcc13','mfcc14','mfcc15','mfcc16','mfcc17','mfcc18','mfcc19']
dataset=pd.DataFrame(columns=columns)
dataseries = pd.Series(data, index = dataset.columns)
dataset = dataset.append(dataseries, ignore_index=True)

example = scaler.transform(np.array(dataset.iloc[:, :], dtype = float))
preds = modelNB.predict(example)
#best_preds = np.asarray([np.argmax(line) for line in preds])

raganame=set()
for i in range(len(raga_list)):
  raganame.add((raga_list[i],y[i]))
raganame=list(raganame)
raganame.sort()
print(preds)
print("Raga for this song is : "+raganame[preds[0]][0])

[1]
Raga for this song is : abhogi


#Random Forest + Preds

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
rf.fit(X_train, Y_train);

In [ ]:
print('Accuracy of RandomForest classifier on training set: {:.2f}'.format(rf.score(X_train, Y_train)))
print('Accuracy of RandomForest classifier on training set: {:.2f}'.format(rf.score(X_test, Y_test)))


Accuracy of RandomForest classifier on training set: 1.00
Accuracy of RandomForest classifier on training set: 1.00


In [ ]:
import librosa
songname='/content/hindolam.mp3'
x, sr = librosa.load(songname, mono=True, duration=30)
rmse = librosa.feature.rmse(y=x)[0]
chroma_stft = librosa.feature.chroma_stft(y=x, sr=sr)
spec_cent = librosa.feature.spectral_centroid(y=x, sr=sr)
spec_bw = librosa.feature.spectral_bandwidth(y=x, sr=sr)
rolloff = librosa.feature.spectral_rolloff(y=x, sr=sr)
zcr = librosa.feature.zero_crossing_rate(x)
mfcc = librosa.feature.mfcc(y=x, sr=sr)

data=[0,np.mean(rmse),np.mean(chroma_stft),np.mean(spec_cent),np.mean(spec_bw),np.mean(rolloff),np.mean(zcr),np.mean(mfcc[0]),np.mean(mfcc[1]),np.mean(mfcc[2]),np.mean(mfcc[3]),np.mean(mfcc[4]),np.mean(mfcc[5]),np.mean(mfcc[6]),np.mean(mfcc[7]),np.mean(mfcc[8]),np.mean(mfcc[9]),np.mean(mfcc[10]),np.mean(mfcc[11]),np.mean(mfcc[12]),np.mean(mfcc[13]),np.mean(mfcc[14]),np.mean(mfcc[15]),np.mean(mfcc[16]),np.mean(mfcc[17]),np.mean(mfcc[18]),np.mean(mfcc[19])]
columns=['ind','rmse','chroma_stft','spec_cent','spec_bw','rolloff','zcr','mfcc0','mfcc1','mfcc2','mfcc3','mfcc4','mfcc5','mfcc6','mfcc7','mfcc8','mfcc9','mfcc10','mfcc11','mfcc12','mfcc13','mfcc14','mfcc15','mfcc16','mfcc17','mfcc18','mfcc19']
dataset=pd.DataFrame(columns=columns)
dataseries = pd.Series(data, index = dataset.columns)
dataset = dataset.append(dataseries, ignore_index=True)

example = scaler.transform(np.array(dataset.iloc[:, :], dtype = float))
preds = rf.predict(example)
best_preds = np.asarray([np.argmax(line) for line in preds])

raganame=set()
for i in range(len(raga_list)):
  raganame.add((raga_list[i],y[i]))
raganame=list(raganame)
raganame.sort()
print(best_preds)
print(preds)
print("Raga for this song is : "+raganame[best_preds[0]][0])

FileNotFoundError: ignored